<a href="https://colab.research.google.com/github/ManabuShinsou/lightweight-gan_SwinIR/blob/main/lightweight_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# lightweight-ganの学習を行うサンプルコード

In [ ]:
# GPU情報の確認
!nvidia-smi

Thu Aug 25 11:46:09 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# google driveからcolaboにファイルをコピーするメソッド
import requests

def download_file_from_google_drive(id, destination):
    print('---- download_file_from_google_drive start ----')
    URL = "https://docs.google.com/uc?export=download"

    try:
        session = requests.Session()

        response = session.get(URL, params = { 'id' : id }, stream = True)
        print('response.status_code: {}'.format(response.status_code))
        print('response.headers: {}'.format(response.headers))

        token = get_confirm_token(response)
        print('token'.format(token))

        if token:
            print(token)
            params = { 'id' : id, 'confirm' : token }
            response = session.get(URL, params = params, stream = True)

        save_response_content(response, destination)
    except Exception as e:
        print(e)


def get_confirm_token(response):
    for key, value in response.cookies.items():
        print('key: {}'.format(key))
        print('value: {}'.format(value))
       
        if key.startswith('download_warning'):
            return value
            
        if key == 'NID':
            print('NID')
            token = value.split('=')
            print(token)
            return token[1]

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [ ]:
# 実行環境のrootディレクトリの確認
# %cd /content/
# %pwd
# !ls -lah

In [ ]:
# Google driveマウント
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# マウントしたGoogle Drive情報の確認
# !ls gdrive -lah

In [ ]:
# datasetを配置するディレクトリを作成
!mkdir ./dataset
# !mkdir ./dataset/inner-generated-100

In [ ]:
# %cd /content/gdrive/MyDrive/'Colab Notebooks'/lightweight-gan/results
# # フォルダを zip 圧縮する
# !zip -r ./inner-generated-100.zip ./inner-generated-100


# # 圧縮した zip ファイルをダウンロードする
# # from google.colab import files
# # files.download("./inner-generated-100.zip")

# %cd /content
# %pwd

In [ ]:
# dataset(zipでgdriveに格納)をColabにコピー
# datasetのサイズによって時間がかかる
%%time
# !cp "./gdrive/MyDrive/dataset/endless_summer.zip" "./dataset"
# !cp "./gdrive/MyDrive/dataset/amime_128_mini.zip" "./dataset"
# !cp "./gdrive/MyDrive/dataset/inner_512.zip" "./dataset"
# !cp "./gdrive/MyDrive/dataset/inner2_512.zip" "./dataset"
# !cp "./gdrive/MyDrive/dataset/inner3_512.zip" "./dataset"
# !cp "./gdrive/MyDrive/dataset/skin_512.zip" "./dataset"
# !cp "./gdrive/MyDrive/dataset/onepiece_512.zip" "./dataset"
!cp "./gdrive/MyDrive/dataset/onepiece_4096.zip" "./dataset"
# !cp "/content/gdrive/MyDrive/Colab Notebooks/lightweight-gan/results/inner-generated-100.zip" "./dataset"

# %cd /content
# %pwd

CPU times: user 60.3 ms, sys: 11.2 ms, total: 71.5 ms
Wall time: 8.38 s


In [ ]:
# コピー(またはダウンロード)したdataset解凍
%%time
# !unzip ./dataset/endless_summer.zip -d ./dataset
# !unzip ./dataset/amime_128_mini.zip -d ./dataset
# !unzip ./dataset/inner_512.zip -d ./dataset
# !unzip ./dataset/inner2_512.zip -d ./dataset
# !unzip ./dataset/inner3_512.zip -d ./dataset
# !unzip ./dataset/skin_512.zip -d ./dataset
# !unzip ./dataset/onepiece_512.zip -d ./dataset
!unzip ./dataset/onepiece_4096.zip -d ./dataset
# !unzip ./dataset/inner-generated-100.zip -d ./dataset

In [ ]:
# データ件数の確認
# !find dataset/* -type f | wc -l

In [ ]:
# Tensorboardの起動
# 学習経過を確認するためにTensorboardを使用します
# --log_dirオプションに各自のGoogle Driveのディレクトリを指定してください。

# %cd /content/
# %load_ext tensorboard
# %tensorboard --logdir=./gdrive/MyDrive/'Colab Notebooks'/gan_sample/chapter7/logs

In [ ]:
# ithgweight-ganのインストールPython
!pip install lightweight-gan==1.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 87 kB 4.5 MB/s 
     |████████████████████████████████| 517 kB 14.9 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=e0c8664c429baacef0323dbc9bf3715be957a34cc6a7fcd1478e8eb95ad80302
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire


In [ ]:
# 作業ディレクトリの移動
%cd /content/gdrive/MyDrive/'Colab Notebooks'/lightweight-gan
%pwd
%ls -al

/content/gdrive/MyDrive/Colab Notebooks/lightweight-gan
total 75
drwx------ 2 root root  4096 Jun 12 07:14 .aim/
drwx------ 2 root root  4096 Jun 12 08:29 inner/
drwx------ 2 root root  4096 Aug 25 11:30 .ipynb_checkpoints/
-rw------- 1 root root 51421 Aug 25 11:46 lightweight-gan.ipynb
drwx------ 2 root root  4096 Jun  2 12:09 models/
drwx------ 2 root root  4096 Jun  2 12:09 results/
drwx------ 2 root root  4096 Aug 25 11:05 SwinIR_wrapper/


# skinの学習（VRoid texture）

In [ ]:
# !lightweight_gan \
#     --data /content/dataset/skin_512 \
#     --image-size 512 \
#     --name skin \
#     --batch-size 16 \
#     --gradient-accumulate-every 4 \
#     --num-train-steps 200000 \
#     --dual-contrast-loss

# onepieceの学習（VRoid texture）

In [ ]:
# !lightweight_gan \
#     --data /content/dataset/onepiece_512 \
#     --name onepiece \
#     --image-size 4096 \
#     --batch-size 16 \
#     --gradient-accumulate-every 4 \
#     --num-train-steps 200000 \
#     --disc-output-size 1 \
#     --aug-types [color] \
#     --dual-contrast-loss


!lightweight_gan \
    --data /content/dataset/onepiece_4096 \
    --name onepiece4096 \
    --batch-size 16 \
    --gradient-accumulate-every 4 \
    --num-train-steps 200000 \
    --aug-types [color] \
    --amp \
    --dual-contrast-loss

autosetting augmentation probability to 30%
onepiece4096</content/dataset/onepiece_4096>:   0% 50/200000 [31:39<2110:34:01, 38.00s/it]G: 23.40 | D: 1.70 | GP: 13.44 | SS: 0.69
onepiece4096</content/dataset/onepiece_4096>:   0% 100/200000 [1:03:20<2110:05:48, 38.00s/it]G: 19.15 | D: 2.00 | GP: 5.01 | SS: 0.61
onepiece4096</content/dataset/onepiece_4096>:   0% 150/200000 [1:34:59<2109:23:43, 38.00s/it]G: 19.07 | D: 2.04 | GP: 19.59 | SS: 0.87
onepiece4096</content/dataset/onepiece_4096>:   0% 200/200000 [2:06:31<2106:39:02, 37.96s/it]G: 23.28 | D: 2.56 | GP: 3.22 | SS: 0.73
onepiece4096</content/dataset/onepiece_4096>:   0% 250/200000 [2:37:57<2103:30:07, 37.91s/it]G: 15.39 | D: 1.87 | GP: 2.92 | SS: 0.72
onepiece4096</content/dataset/onepiece_4096>:   0% 300/200000 [3:09:32<2102:46:26, 37.91s/it]G: 21.84 | D: 1.10 | GP: 4.63 | SS: 0.64
onepiece4096</content/dataset/onepiece_4096>:   0% 350/200000 [3:41:42<2107:49:13, 38.01s/it]G: 13.66 | D: 3.32 | GP: 3.10 | SS: 0.66
onepiece4096</conte

# innerの学習（VRoid texture）

In [ ]:
# 学習の実行
# ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★
# ★★★★★　長時間の学習を行うとGoogle Driveの容量が足りなくなる事があるので注意!!!!!!　★★★★★
# ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★
# Google Driveの容量が少ない方は事前に容量を購入

# -–data: 学習用data setのディレクトリ
# -–name: 学習したモデルや画像の保存先
# -–batch-size: バッチサイズ


# !lightweight_gan \
#     --data /content/dataset/inner3_512 \
#     --image-size 512 \
#     --name inner \
#     --batch-size 16 \
#     --gradient-accumulate-every 4 \
#     --num-train-steps 200000 \
#     --aug-types [color] \
#     --dual-contrast-loss

    # --aug-test \
    # --dual-contrast-loss \
    # --aug-types [color,cutout,offset,translation] \
    # --show-progress \
    # --num-image-tiles 2
    # --amp

# 学習済みモデルからGIFを作成

In [ ]:
!pip install aim
# !pip install lightweight-gan
!pip install lightweight-gan==1.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.4 MB 5.4 MB/s 
     |████████████████████████████████| 3.1 MB 35.2 MB/s 
     |████████████████████████████████| 4.1 MB 48.3 MB/s 
     |████████████████████████████████| 6.1 MB 34.7 MB/s 
     |████████████████████████████████| 51 kB 483 kB/s 
     |████████████████████████████████| 18.6 MB 1.2 MB/s 
     |████████████████████████████████| 57 kB 6.3 MB/s 
     |████████████████████████████████| 281 kB 77.5 MB/s 
     |████████████████████████████████| 209 kB 75.0 MB/s 
     |████████████████████████████████| 55 kB 4.8 MB/s 
     |████████████████████████████████| 78 kB 9.6 MB/s 
     |████████████████████████████████| 60 kB 7.7 MB/s 
     |████████████████████████████████| 58 kB 7.6 MB/s 
  Created wheel for aim-ui: filename=aim_ui-3.13.0-py3-none-any.whl size=18735991 sha256=5166e73026e1fcb7962cebb700f6bcb5a1aeedc3528ec944e5c33b2684225550
  Stor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# 作業ディレクトリの移動
%cd /content/gdrive/MyDrive/'Colab Notebooks'/lightweight-gan

/content/gdrive/MyDrive/Colab Notebooks/lightweight-gan


In [ ]:
# 学習済みモデルからGIFを作成
# !lightweight_gan \
#     --name /content/gdrive/MyDrive/'Colab Notebooks'/lightweight-gan/inner \
#     --generate-interpolation

!lightweight_gan \
    --name inner \
    --load-from 300 \
    --image-size 1024 \
    --generate \
    --num-image-tiles 10

# !lightweight_gan \
#     --name inner \
#     --load-from 100 \
#     --show-progress \
#     --num-image-tiles 10

The lock file /content/gdrive/MyDrive/Colab Notebooks/lightweight-gan/.aim/.repo_lock is on a filesystem of type `fuse.drive` (device id: 36). Using soft file locks to avoid potential data corruption.
loading from version 1.0.0
Saving generated default images: 100% 10/10 [00:07<00:00,  1.29it/s]
Saving generated EMA images: 100% 10/10 [00:01<00:00,  8.71it/s]
sample images generated at /content/gdrive/MyDrive/Colab Notebooks/lightweight-gan/results/inner-generated-300


# upload files

In [ ]:
# 作業ディレクトリの移動
%cd /content
%ls -al

/content
total 24
drwxr-xr-x 1 root root 4096 Aug 25 11:46 ./
drwxr-xr-x 1 root root 4096 Aug 25 11:44 ../
drwxr-xr-x 4 root root 4096 Aug 15 13:44 .config/
drwxr-xr-x 2 root root 4096 Aug 25 11:46 dataset/
drwx------ 5 root root 4096 Aug 25 11:46 gdrive/
drwxr-xr-x 1 root root 4096 Aug 15 13:44 sample_data/


In [ ]:
# copy file from Google Drive to Google Colab
%cd /content/gdrive/My\ Drive/Colab\ Notebooks/lightweight-gan/results
!zip -r ./import.zip ./inner-generated-300 
# !zip -r /content/gdrive/My\ Drive/Colab\ Notebooks/lightweight-gan/results/import.zip /content/gdrive/My\ Drive/Colab\ Notebooks/lightweight-gan/results/inner-generated-300 
# !cp /content/drive/My\ Drive/dataset/inner3_512.zip ./dataset

%cd /content
# !mkdir ./dataset
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/lightweight-gan/results/import.zip ./dataset
# !unzip ./dataset/inner3_512.zip
!unzip ./dataset/import.zip -d ./dataset


# Wrapper for SwinIR

> Demo notebook for [__Wrapper for SwinIR__](https://github.com/Lin-Sinorodin/SwinIR_wrapper) repository. This wrapper based on the official PyTorch implementation of [SwinIR: Image Restoration Using Shifted Window Transformer](https://arxiv.org/abs/2108.10257). Check the repo for more details.

In [ ]:
#@title Imports and Utils { display-mode: "form" }

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
%config InlineBackend.rc = {'figure.figsize': (10.0, 10.0)}

!git clone -qq https://github.com/Lin-Sinorodin/SwinIR_wrapper.git
!pip install -qq timm
from SwinIR_wrapper.SwinIR_wrapper import SwinIR_SR

import cv2
import torch
import urllib.request
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available():
    print(f'Using GPU: {torch.cuda.get_device_properties(0).name}')
else:
    print('Using CPU. Concider using GPU for faster inference.')

def compare_sr_with_original(img_lq, img_hq):
    plt.figure()

    plt.subplot(1, 2, 1)
    plt.imshow(img_lq[::,::,::-1])
    plt.title(f'Original - {img_lq.shape}')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(img_hq[::,::,::-1])
    plt.title(f'Super Resolution - {img_hq.shape}')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

fatal: destination path 'SwinIR_wrapper' already exists and is not an empty directory.
Using GPU: Tesla T4


# FILE DIRECTORY

In [ ]:
# DIR_FILE = "/content/dataset/inner-generated-300/generated-08-25-2022_11-02-31-2.jpg"
# DIR_FILE = "/content/dataset/inner-generated-300/generated-08-25-2022_11-02-31-9.jpg"
# DIR_FILE = "/content/dataset/inner-generated-300/generated-08-25-2022_11-47-27-1.jpg"
# DIR_FILE = "/content/dataset/inner-generated-300/generated-08-25-2022_11-47-27-6-ema.jpg"
# DIR_FILE = "/content/dataset/inner-generated-300/generated-08-25-2022_11-47-27-8.jpg"
DIR_FILE = "/content/dataset/inner-generated-300/generated-08-25-2022_11-47-27-3-ema.jpg"

In [ ]:
# SwinIR
get_image_method = "upload file" #["upload file", "download from url"]

uploaded_file = DIR_FILE
if get_image_method == 'upload file': 
    path = uploaded_file

url = DIR_FILE 
if get_image_method == 'download from url':
    path = url.split('/')[-1]
    urllib.request.urlretrieve(url, path)

pretrained_model = "real_sr x4" #["real_sr x4", "classical_sr x2", "classical_sr x3", "classical_sr x4", "classical_sr x8", "lightweight x2", "lightweight x3", "lightweight x4"]

model_type, scale = pretrained_model.split(' ')
scale = int(scale[1])

# initialize super resolution model
sr = SwinIR_SR(model_type, scale)

print(f'Loaded {pretrained_model} successfully')

# load the test image from previus step
img_lq = cv2.imread(path, cv2.IMREAD_COLOR)

# feed the image to the SR model
img_hq = sr.upscale_using_patches(img_lq, slice_dim=64, keep_pbar=True)

# show results
# compare_sr_with_original(img_lq, img_hq)

# save img
import cv2
import numpy as np

# print(img_hq.dtype)
# img_hq

img = img_hq # Your image as a numpy array 

cv2.imwrite("out.png", img)

# # copy file from Google Colab to Google Drive
# # !zip -r sample.zip sample 
# # !cp logs/sample.zip /content/drive/My\ Drive/sample.zip
# !zip -r out.zip out.png
# !cp /content/out.npz /content/drive/My\ Drive/Colab\ Notebooks/StableDiffusion/out.npz

# !zip -r out.zip out.png 
# !cp /content/out.zip /content/gdrive/My\ Drive/Colab\ Notebooks/lightweight-gan/results/SwinIR/out.zip
# !unzip /content/gdrive/My\ Drive/Colab\ Notebooks/lightweight-gan/results/SwinIR/out.zip -d /content/gdrive/My\ Drive/Colab\ Notebooks/lightweight-gan/results/SwinIR

# download Google Colab to PC
from google.colab import files
files.download('out.png')

Loaded real_sr x4 successfully


Performing SR on patches:   0%|          | 0/256 [00:00<?, ?patch/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# # SwinIR
# get_image_method = "upload file" #["upload file", "download from url"]

# uploaded_file = DIR_FILE
# if get_image_method == 'upload file': 
#     path = uploaded_file

# url = DIR_FILE 
# if get_image_method == 'download from url':
#     path = url.split('/')[-1]
#     urllib.request.urlretrieve(url, path)

# pretrained_model = "real_sr x4" #["real_sr x4", "classical_sr x2", "classical_sr x3", "classical_sr x4", "classical_sr x8", "lightweight x2", "lightweight x3", "lightweight x4"]

# model_type, scale = pretrained_model.split(' ')
# scale = int(scale[1])

# # initialize super resolution model
# sr = SwinIR_SR(model_type, scale)

# print(f'Loaded {pretrained_model} successfully')

# Get Test Image

In [ ]:
#@title Get Test Image { run: "auto", display-mode: "form" }

#@markdown > #### Choose between uploading image or downloading from url:
get_image_method = "upload file" #@param ["upload file", "download from url"]

#@markdown * for uploaded image, write it's path:
uploaded_file = DIR_FILE #@param {type:"string"}
if get_image_method == 'upload file':
    path = uploaded_file

#@markdown * for url download, provide the image url:
url = DIR_FILE #@param {type:"string"}
if get_image_method == 'download from url':
    path = url.split('/')[-1]
    urllib.request.urlretrieve(url, path)

In [ ]:
#@title Setup Super Resolution Model { run: "auto" }
pretrained_model = "real_sr x4" #@param ["real_sr x4", "classical_sr x2", "classical_sr x3", "classical_sr x4", "classical_sr x8", "lightweight x2", "lightweight x3", "lightweight x4"]

model_type, scale = pretrained_model.split(' ')
scale = int(scale[1])

# initialize super resolution model
sr = SwinIR_SR(model_type, scale)

print(f'Loaded {pretrained_model} successfully')

Loaded real_sr x4 successfully


# Use The Model

> If the image size isn't too big, the image can be fed to the model as it is However, for larger images the GPU will run out of memory. A simple solution is to use the model on smaller patches and then combine the results for a large image.

### Directly - `sr.upscale()`


In [ ]:
# # load the test image from previus step
# img_lq = cv2.imread(path, cv2.IMREAD_COLOR)

# # feed the image to the SR model
# img_hq = sr.upscale(img_lq)                  

# # show results
# compare_sr_with_original(img_lq, img_hq)     

### On Patches - `sr.upscale_using_patches()`

In [ ]:
# load the test image from previus step
img_lq = cv2.imread(path, cv2.IMREAD_COLOR)

# feed the image to the SR model
img_hq = sr.upscale_using_patches(img_lq, slice_dim=64, keep_pbar=True)

# show results
compare_sr_with_original(img_lq, img_hq)

In [ ]:
import cv2
import numpy as np

# print(img_hq.dtype)
# img_hq

img = img_hq # Your image as a numpy array 

cv2.imwrite("out.png", img)

True

In [ ]:
# import matplotlib.image
# matplotlib.image.imsave('out.png', img_hq)

# # img_hq.save(f"out.png")
# from PIL import Image
# im = Image.fromarray(img_hq)
# im.save("out.jpeg")

In [ ]:
# # resize
# from PIL import Image

# img = Image.open('./out.png')

# img_resize = img.resize((2048, 2048))
# img_resize.save('./out_re.png')

# img_resize_lanczos = img.resize((2048, 2048), Image.LANCZOS)
# img_resize_lanczos.save('./out_lanczos.jpg')

# OUTPUT FILE

In [ ]:
# # copy file from Google Colab to Google Drive
# # !zip -r sample.zip sample 
# # !cp logs/sample.zip /content/drive/My\ Drive/sample.zip
# !zip -r out.zip out.png
# !cp /content/out.npz /content/drive/My\ Drive/Colab\ Notebooks/StableDiffusion/out.npz

!zip -r out.zip out.png 
!cp /content/out.zip /content/gdrive/My\ Drive/Colab\ Notebooks/lightweight-gan/results/SwinIR/out.zip
!unzip /content/gdrive/My\ Drive/Colab\ Notebooks/lightweight-gan/results/SwinIR/out.zip -d /content/gdrive/My\ Drive/Colab\ Notebooks/lightweight-gan/results/SwinIR

# download Google Colab to PC
from google.colab import files
files.download('out.png')

updating: out.png (deflated 6%)
Archive:  /content/gdrive/My Drive/Colab Notebooks/lightweight-gan/results/SwinIR/out.zip
replace /content/gdrive/My Drive/Colab Notebooks/lightweight-gan/results/SwinIR/out.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: r
new name: out6
  inflating: /content/gdrive/My Drive/Colab Notebooks/lightweight-gan/results/SwinIR/out6  


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>